In [1]:
train_size = 47500

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import time
from sklearn.model_selection import cross_validate
%matplotlib inline

In [3]:
np.set_printoptions(precision=9)
np.set_printoptions(linewidth=np.inf)

In [5]:
X = np.load('../../X_train.npz')['arr_0']
Y = np.load('../../Y_train.npz')['arr_0']
TX = np.load('../../X_test.npz')['arr_0']

In [6]:
rd =  np.random.permutation(47500)
X, Y = X[rd], Y[rd]

In [7]:
XTX = np.concatenate((X, TX), axis=0)

In [8]:
XTX_square = XTX ** 2
XTX_cumsum = np.concatenate((np.cumsum(XTX[:, :5000], axis=1), np.cumsum(XTX[:, 5000:], axis=1)), axis=1)
XTX_all = np.concatenate((XTX, XTX_square, XTX_cumsum), axis=1)

In [11]:
XTX_all.shape

(50000, 30000)

In [10]:
# calculate type 1 error
def err1(y, y_pred):
    return np.sum(1.0 * np.abs(y_pred - y)) / len(y_pred)

# calculate type 2 errr
def err2(y, y_pred):
    return np.sum(np.abs(y_pred - y) / y) / len(y_pred)

In [12]:
y_id = 0

gbm0 = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=1000,
                        n_jobs=-1)

t = time.time()
gbm0.fit(XTX_all[:42500], Y[:42500, y_id], 
        eval_set=[(XTX_all[42500:45000], Y[42500:45000, y_id])], 
        eval_metric='l1', 
        early_stopping_rounds=5)
print(time.time() - t)

print('in:')
print(300.0 * err1(gbm0.predict(XTX_all[:42500]), Y[:42500, y_id]))
print(err2(gbm0.predict(XTX_all[:42500]), Y[:42500, y_id]))

print('val:')
print(300.0 * err1(gbm0.predict(XTX_all[42500:45000]), Y[42500:45000, y_id]))
print(err2(gbm0.predict(XTX_all[42500:45000]), Y[42500:45000, y_id]))

print('test:')
print(300.0 * err1(gbm0.predict(XTX_all[45000:47500]), Y[45000:47500, y_id]))
print(err2(gbm0.predict(XTX_all[45000:47500]), Y[45000:47500, y_id]))

ty0 = gbm0.predict(XTX_all[47500:])

[1]	valid_0's l1: 0.245267	valid_0's l2: 0.0792403
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.238125	valid_0's l2: 0.0745798
[3]	valid_0's l1: 0.231058	valid_0's l2: 0.0702576
[4]	valid_0's l1: 0.224512	valid_0's l2: 0.0663048
[5]	valid_0's l1: 0.218395	valid_0's l2: 0.0627748
[6]	valid_0's l1: 0.212505	valid_0's l2: 0.0594823
[7]	valid_0's l1: 0.20686	valid_0's l2: 0.0564405
[8]	valid_0's l1: 0.201582	valid_0's l2: 0.0537131
[9]	valid_0's l1: 0.196653	valid_0's l2: 0.0512071
[10]	valid_0's l1: 0.192009	valid_0's l2: 0.0489328
[11]	valid_0's l1: 0.1877	valid_0's l2: 0.0468202
[12]	valid_0's l1: 0.183471	valid_0's l2: 0.044883
[13]	valid_0's l1: 0.179734	valid_0's l2: 0.0431675
[14]	valid_0's l1: 0.176103	valid_0's l2: 0.041552
[15]	valid_0's l1: 0.172574	valid_0's l2: 0.0400189
[16]	valid_0's l1: 0.169215	valid_0's l2: 0.0386249
[17]	valid_0's l1: 0.166132	valid_0's l2: 0.0373514
[18]	valid_0's l1: 0.163096	valid_0's l2: 0.0361751
[19]	valid_0's l1

[157]	valid_0's l1: 0.0991067	valid_0's l2: 0.0178292
[158]	valid_0's l1: 0.0990991	valid_0's l2: 0.017821
[159]	valid_0's l1: 0.0990527	valid_0's l2: 0.0178121
[160]	valid_0's l1: 0.0990549	valid_0's l2: 0.0178131
[161]	valid_0's l1: 0.0990333	valid_0's l2: 0.0178116
[162]	valid_0's l1: 0.0989996	valid_0's l2: 0.0178006
[163]	valid_0's l1: 0.0989895	valid_0's l2: 0.0178001
[164]	valid_0's l1: 0.0989886	valid_0's l2: 0.0178005
[165]	valid_0's l1: 0.0989766	valid_0's l2: 0.0177985
[166]	valid_0's l1: 0.0989825	valid_0's l2: 0.0178039
[167]	valid_0's l1: 0.0989501	valid_0's l2: 0.0177954
[168]	valid_0's l1: 0.0989798	valid_0's l2: 0.0178057
[169]	valid_0's l1: 0.0989634	valid_0's l2: 0.0177989
[170]	valid_0's l1: 0.0989066	valid_0's l2: 0.0177825
[171]	valid_0's l1: 0.0989075	valid_0's l2: 0.0177835
[172]	valid_0's l1: 0.0988768	valid_0's l2: 0.0177738
[173]	valid_0's l1: 0.0988789	valid_0's l2: 0.0177752
[174]	valid_0's l1: 0.0988689	valid_0's l2: 0.01777
[175]	valid_0's l1: 0.0988654	v

In [13]:
y_id = 1

gbm1 = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=1000,
                        n_jobs=-1)

t = time.time()
gbm1.fit(XTX_all[:42500], Y[:42500, y_id], 
        eval_set=[(XTX_all[42500:45000], Y[42500:45000, y_id])], 
        eval_metric='l1', 
        early_stopping_rounds=5)
print(time.time() - t)

print('in:')
print(1.0 * err1(gbm1.predict(XTX_all[:42500]), Y[:42500, y_id]))
print(err2(gbm1.predict(XTX_all[:42500]), Y[:42500, y_id]))

print('val:')
print(1.0 * err1(gbm1.predict(XTX_all[42500:45000]), Y[42500:45000, y_id]))
print(err2(gbm1.predict(XTX_all[42500:45000]), Y[42500:45000, y_id]))

print('test:')
print(1.0 * err1(gbm1.predict(XTX_all[45000:47500]), Y[45000:47500, y_id]))
print(err2(gbm1.predict(XTX_all[45000:47500]), Y[45000:47500, y_id]))

ty1 = gbm1.predict(XTX_all[47500:])

[1]	valid_0's l1: 54.7859	valid_0's l2: 3964.14
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 53.0767	valid_0's l2: 3722.5
[3]	valid_0's l1: 51.4715	valid_0's l2: 3503.51
[4]	valid_0's l1: 49.9415	valid_0's l2: 3301.41
[5]	valid_0's l1: 48.5052	valid_0's l2: 3119.9
[6]	valid_0's l1: 47.1312	valid_0's l2: 2951.61
[7]	valid_0's l1: 45.8256	valid_0's l2: 2798.23
[8]	valid_0's l1: 44.5647	valid_0's l2: 2654.81
[9]	valid_0's l1: 43.3958	valid_0's l2: 2526.36
[10]	valid_0's l1: 42.2609	valid_0's l2: 2406.21
[11]	valid_0's l1: 41.199	valid_0's l2: 2296.13
[12]	valid_0's l1: 40.2342	valid_0's l2: 2196.94
[13]	valid_0's l1: 39.3172	valid_0's l2: 2104.89
[14]	valid_0's l1: 38.4053	valid_0's l2: 2018.1
[15]	valid_0's l1: 37.5863	valid_0's l2: 1941.6
[16]	valid_0's l1: 36.766	valid_0's l2: 1868.64
[17]	valid_0's l1: 35.9713	valid_0's l2: 1799.35
[18]	valid_0's l1: 35.2136	valid_0's l2: 1736.23
[19]	valid_0's l1: 34.5906	valid_0's l2: 1683.72
[20]	valid_0's l1: 33.9

[167]	valid_0's l1: 20.2664	valid_0's l2: 785.9
[168]	valid_0's l1: 20.2588	valid_0's l2: 785.58
[169]	valid_0's l1: 20.2576	valid_0's l2: 785.257
[170]	valid_0's l1: 20.241	valid_0's l2: 784.374
[171]	valid_0's l1: 20.2299	valid_0's l2: 783.949
[172]	valid_0's l1: 20.2226	valid_0's l2: 783.891
[173]	valid_0's l1: 20.2217	valid_0's l2: 783.728
[174]	valid_0's l1: 20.221	valid_0's l2: 783.764
[175]	valid_0's l1: 20.2201	valid_0's l2: 783.608
[176]	valid_0's l1: 20.2218	valid_0's l2: 783.535
[177]	valid_0's l1: 20.2145	valid_0's l2: 782.984
[178]	valid_0's l1: 20.2097	valid_0's l2: 782.581
[179]	valid_0's l1: 20.2057	valid_0's l2: 782.248
[180]	valid_0's l1: 20.2012	valid_0's l2: 781.939
[181]	valid_0's l1: 20.2046	valid_0's l2: 782.351
[182]	valid_0's l1: 20.2033	valid_0's l2: 782.209
[183]	valid_0's l1: 20.2023	valid_0's l2: 782.055
[184]	valid_0's l1: 20.1989	valid_0's l2: 781.694
[185]	valid_0's l1: 20.2001	valid_0's l2: 781.822
[186]	valid_0's l1: 20.1939	valid_0's l2: 781.869
[187]

In [14]:
y_id = 2

gbm2 = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=1000,
                        n_jobs=-1)

t = time.time()
gbm2.fit(XTX_all[:42500], Y[:42500, y_id], 
        eval_set=[(XTX_all[42500:45000], Y[42500:45000, y_id])], 
        eval_metric='l1', 
        early_stopping_rounds=5)
print(time.time() - t)

print('in:')
print(200.0 * err1(gbm2.predict(XTX_all[:42500]), Y[:42500, y_id]))
print(err2(gbm2.predict(XTX_all[:42500]), Y[:42500, y_id]))

print('val:')
print(200.0 * err1(gbm2.predict(XTX_all[42500:45000]), Y[42500:45000, y_id]))
print(err2(gbm2.predict(XTX_all[42500:45000]), Y[42500:45000, y_id]))

print('test:')
print(200.0 * err1(gbm2.predict(XTX_all[45000:47500]), Y[45000:47500, y_id]))
print(err2(gbm2.predict(XTX_all[45000:47500]), Y[45000:47500, y_id]))

ty2 = gbm2.predict(XTX_all[47500:])

[1]	valid_0's l1: 0.118558	valid_0's l2: 0.0188638
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.112683	valid_0's l2: 0.0170525
[3]	valid_0's l1: 0.107109	valid_0's l2: 0.015418
[4]	valid_0's l1: 0.101819	valid_0's l2: 0.0139419
[5]	valid_0's l1: 0.0968047	valid_0's l2: 0.0126109
[6]	valid_0's l1: 0.0920243	valid_0's l2: 0.0114042
[7]	valid_0's l1: 0.0874948	valid_0's l2: 0.0103182
[8]	valid_0's l1: 0.0831764	valid_0's l2: 0.00933418
[9]	valid_0's l1: 0.0790999	valid_0's l2: 0.00844901
[10]	valid_0's l1: 0.0752118	valid_0's l2: 0.00764681
[11]	valid_0's l1: 0.0715404	valid_0's l2: 0.00692537
[12]	valid_0's l1: 0.0680356	valid_0's l2: 0.00627096
[13]	valid_0's l1: 0.0647125	valid_0's l2: 0.00568068
[14]	valid_0's l1: 0.0615586	valid_0's l2: 0.0051477
[15]	valid_0's l1: 0.0585715	valid_0's l2: 0.00466623
[16]	valid_0's l1: 0.0557225	valid_0's l2: 0.00423027
[17]	valid_0's l1: 0.0530192	valid_0's l2: 0.00383649
[18]	valid_0's l1: 0.0504521	valid_0's l2: 

[148]	valid_0's l1: 0.00677453	valid_0's l2: 8.07658e-05
[149]	valid_0's l1: 0.00677263	valid_0's l2: 8.07534e-05
[150]	valid_0's l1: 0.00676949	valid_0's l2: 8.07051e-05
[151]	valid_0's l1: 0.00676531	valid_0's l2: 8.05791e-05
[152]	valid_0's l1: 0.00676109	valid_0's l2: 8.04855e-05
[153]	valid_0's l1: 0.00675995	valid_0's l2: 8.04602e-05
[154]	valid_0's l1: 0.0067568	valid_0's l2: 8.04276e-05
[155]	valid_0's l1: 0.0067553	valid_0's l2: 8.03973e-05
[156]	valid_0's l1: 0.0067514	valid_0's l2: 8.03105e-05
[157]	valid_0's l1: 0.00674889	valid_0's l2: 8.02087e-05
[158]	valid_0's l1: 0.00674617	valid_0's l2: 8.01581e-05
[159]	valid_0's l1: 0.00674452	valid_0's l2: 8.00984e-05
[160]	valid_0's l1: 0.00674181	valid_0's l2: 8.00572e-05
[161]	valid_0's l1: 0.00674052	valid_0's l2: 8.0023e-05
[162]	valid_0's l1: 0.00673986	valid_0's l2: 8.00333e-05
[163]	valid_0's l1: 0.00673619	valid_0's l2: 7.99795e-05
[164]	valid_0's l1: 0.00673277	valid_0's l2: 7.99164e-05
[165]	valid_0's l1: 0.00672965	vali

In [15]:
ty = np.array([ty0, ty1, ty2]).transpose()
print(ty.shape)
df = pd.DataFrame(ty)
df.to_csv('y_test.csv', index=False, header=False)

(2500, 3)


In [16]:
Ein1 = 26.344982331115254 + 17.45640179299457 + 1.156490476051319
Ein2 = 0.18244789284412063 + 0.12947610829979372 + 0.0080620821133244
Eval1 = 29.61047625565683 + 20.090015293496176 + 1.3296296264559126
Eval2 = 0.26212510103760256 + 0.14765436973722826 + 0.009333481247403613
Etest1 = 29.486975438346228 + 20.515535622666953 + 1.348832756366387
Etest2 = 0.36599250712064635 + 0.14940815314278233 + 0.009403894489712772

In [18]:
print(Ein1, Ein2, Eval1, Eval2, Etest1, Etest2)

44.95787460016114 0.31998608325723876 51.03012117560892 0.4191129520222344 51.351343817379565 0.5248045547531415
